<a href="https://colab.research.google.com/github/xtt001/PM-Jobs/blob/main/Elc_Price_Cal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install scipy

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

# ==========================================
# 第一部分：数据清洗与提取工具
# ==========================================
def load_and_clean_data(file_path, price_col_index=2):
    """
    读取混乱的CSV文件，提取时间与指定地区的电价。

    参数:
    file_path: 文件路径
    price_col_index: 价格数据所在的列索引（从0开始计数）。
                     根据你的截图：
                     0 = Start date
                     1 = End date
                     2 = Germany/Luxembourg (目标数据)
                     如果要提取其他地区，只需修改这个数字。
    """
    print(f"正在读取文件: {file_path} ...")

    try:
        # 1. 读取文件，指定分隔符为分号 ';'
        # on_bad_lines='skip' 用于跳过可能存在的坏行
        df = pd.read_csv(file_path, sep=';', on_bad_lines='skip')

        # 2. 提取必要的列
        # 获取第一列（时间）和 指定的电价列
        # iloc[:, [0, price_col_index]] 表示取所有行的第0列和第price_col_index列
        target_col_name = df.columns[price_col_index]
        print(f"正在提取数据列: {target_col_name}")

        clean_df = df.iloc[:, [0, price_col_index]].copy()
        clean_df.columns = ['RawTime', 'Price'] # 重命名方便后续处理

        # 3. 清洗时间格式
        # 你的格式是 "31-Oct 2025 12:00 AM"
        # 我们让 pandas 自动推断格式，同时处理可能出现的空值
        clean_df['Datetime'] = pd.to_datetime(clean_df['RawTime'], errors='coerce')

        # 删除时间解析失败的行（比如最后可能的空行）
        clean_df = clean_df.dropna(subset=['Datetime'])

        # 4. 清洗价格数据
        # 有时候欧洲数据会把小数点写成逗号（如 87,74），这里做个防御性处理
        if clean_df['Price'].dtype == object:
            clean_df['Price'] = clean_df['Price'].astype(str).str.replace(',', '.')

        # 强制转为数字，无法转换的变为 NaN
        clean_df['Price'] = pd.to_numeric(clean_df['Price'], errors='coerce')

        # 删除没有价格数据的行
        clean_df = clean_df.dropna(subset=['Price'])

        # 设置时间为索引并排序
        clean_df = clean_df.set_index('Datetime').sort_index()

        print(f"数据清洗完成！共提取 {len(clean_df)} 条有效记录。\n")
        return clean_df

    except Exception as e:
        print(f"❌ 读取或清洗数据时发生错误: {e}")
        return None

# ==========================================
# 第二部分：波峰波谷分析逻辑
# ==========================================
def analyze_daily_peaks(clean_df, prominence_threshold=10):
    """
    按天分组，分析每一天的波峰波谷
    """
    # 按日期分组
    grouped = clean_df.groupby(clean_df.index.date)

    for date, day_data in grouped:
        print(f"\n{'='*20} 分析日期: {date} {'='*20}")

        # 提取当前天的数据
        prices = day_data['Price'].values
        times = day_data.index.strftime('%H:%M').tolist()

        # 确保数据量足够（比如一天至少要有10个小时的数据）
        if len(prices) < 10:
            print("数据不足，跳过此日。")
            continue

        # --- 核心算法 ---
        # 寻找波峰 (距离设为2，即2小时内不重复算波峰)
        peaks_indices, _ = find_peaks(prices, prominence=prominence_threshold, distance=2)
        # 寻找波谷
        valleys_indices, _ = find_peaks(-prices, prominence=prominence_threshold, distance=2)

        # --- 输出结果 ---
        print(f"【波峰 (高价/建议卖电)】:")
        if len(peaks_indices) == 0:
            print("  (无显著波峰)")
        for i in peaks_indices:
            print(f"  ⏰ {times[i]} \t 💰 {prices[i]:.2f}")

        print(f"-"*30)

        print(f"【波谷 (低价/建议充电)】:")
        if len(valleys_indices) == 0:
            print("  (无显著波谷)")
        for i in valleys_indices:
            print(f"  ⏰ {times[i]} \t 💰 {prices[i]:.2f}")

        # (可选) 如果你想每天都弹出一张图，把下面的注释打开
        # plot_day(day_data, peaks_indices, valleys_indices, date)

def plot_day(day_data, peaks, valleys, date_label):
    """画单日的图"""
    prices = day_data['Price'].values
    times = day_data.index.strftime('%H:%M')

    plt.figure(figsize=(10, 4))
    plt.plot(times, prices, label='Price', color='#1f77b4')
    plt.plot(peaks, prices[peaks], "x", color='red', markersize=10, label='Peak')
    plt.plot(valleys, prices[valleys], "x", color='green', markersize=10, label='Valley')
    plt.title(f'Electricity Prices: {date_label}')
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# ==========================================
# 主执行区
# ==========================================
if __name__ == "__main__":
    # 1. 设置文件路径 (请修改为你实际的文件名)
    # 注意：如果是Windows路径，建议在字符串前加 r，如 r"C:\Users\Downloads\data.csv"
    csv_file_path = "/content/data.csv"

    # 2. 清洗数据
    # price_col_index=2 代表提取第3列数据（即 Germany/Luxembourg）
    # 如果你想提取后面的国家，比如 'Denmark 1'，数一下它在第几列，改这个数字即可
    df_clean = load_and_clean_data(csv_file_path, price_col_index=2)

    # 3. 开始分析
    if df_clean is not None:
        # prominence_threshold: 价格突出的阈值。
        # 如果你觉得捕捉到的波峰太多太杂，把这个数字调大（比如 15 或 20）
        analyze_daily_peaks(df_clean, prominence_threshold=10)

In [ ]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks

# ==========================================
# 1. 数据清洗 (沿用之前的逻辑，稍作精简)
# ==========================================
def load_and_clean_data(file_path, price_col_index=2):
    try:
        # 读取CSV，处理分号分隔符
        df = pd.read_csv(file_path, sep=';', on_bad_lines='skip')

        # 提取时间列和指定的价格列
        clean_df = df.iloc[:, [0, price_col_index]].copy()
        clean_df.columns = ['RawTime', 'Price']

        # 清洗时间
        clean_df['Datetime'] = pd.to_datetime(clean_df['RawTime'], errors='coerce')
        clean_df = clean_df.dropna(subset=['Datetime'])

        # 清洗价格 (处理逗号小数点)
        if clean_df['Price'].dtype == object:
            clean_df['Price'] = clean_df['Price'].astype(str).str.replace(',', '.')
        clean_df['Price'] = pd.to_numeric(clean_df['Price'], errors='coerce')
        clean_df = clean_df.dropna(subset=['Price'])

        # 设置索引并排序
        clean_df = clean_df.set_index('Datetime').sort_index()
        return clean_df
    except Exception as e:
        print(f"数据读取错误: {e}")
        return None

# ==========================================
# 2. 核心业务逻辑：计算每日最佳充放电策略
# ==========================================
def calculate_daily_strategy(clean_df, battery_kwh=3, prominence=5):
    """
    输入: 清洗后的DataFrame
    输出: 包含每日策略的列表
    """
    results = []

    # 按天分组
    grouped = clean_df.groupby(clean_df.index.date)

    for date, day_data in grouped:
        prices = day_data['Price'].values
        times = day_data.index

        # 如果数据点太少，跳过
        if len(prices) < 5:
            continue

        # 1. 寻找当天的所有波峰和波谷
        # distance=2 保证波峰波谷不会粘连太近
        peak_indices, _ = find_peaks(prices, prominence=prominence, distance=2)
        valley_indices, _ = find_peaks(-prices, prominence=prominence, distance=2)

        # 2. 寻找最佳匹配对 (暴力搜索最优解)
        # 逻辑：必须先充(Valley)后放(Peak)，且收益最大

        max_profit = -float('inf')
        best_strategy = None # 存储 (充电时间, 充电价, 放电时间, 放电价)

        # 遍历每一个波谷作为可能的“充电点”
        for v_idx in valley_indices:
            buy_price = prices[v_idx]

            # 找到在这个波谷时间点“之后”出现的所有波峰
            valid_peaks = [p_idx for p_idx in peak_indices if p_idx > v_idx]

            if not valid_peaks:
                continue # 这个波谷后面没有波峰了，没法卖电

            # 在这些合法的波峰中，找价格最高的那个卖
            best_p_idx = max(valid_peaks, key=lambda i: prices[i])
            sell_price = prices[best_p_idx]

            # 计算这一对的单价差
            spread = sell_price - buy_price

            # 更新最大利润记录
            if spread > max_profit:
                max_profit = spread
                best_strategy = {
                    'Date': date,
                    'Charge_Time': times[v_idx].strftime('%H:%M'),
                    'Charge_Price': buy_price,
                    'Discharge_Time': times[best_p_idx].strftime('%H:%M'),
                    'Discharge_Price': sell_price,
                    'Price_Spread': spread,
                    'Total_Profit': spread/1000 * battery_kwh # 乘以电池容量
                }

        # 3. 如果当天找到了有利可图的策略 (且利润大于0)
        if best_strategy and best_strategy['Total_Profit'] > 0:
            results.append(best_strategy)
        else:
            # 如果当天没有操作空间（单边下跌或波动极小），也可以记录一条空数据或者跳过
            results.append({
                'Date': date,
                'Charge_Time': 'No Action', 'Charge_Price': 0,
                'Discharge_Time': '-', 'Discharge_Price': 0,
                'Price_Spread': 0, 'Total_Profit': 0
            })

    return pd.DataFrame(results)

# ==========================================
# 3. 主程序
# ==========================================
if __name__ == "__main__":
    # 配置部分
    input_file = "data.csv"          # 输入文件名
    output_file = "/content/battery_profit_report.csv" # 输出文件名
    battery_capacity = 3             # 电池容量 3度电

    print("--- 开始处理 ---")

    # 1. 清洗数据
    # price_col_index=2 对应 Germany/Luxembourg
    df = load_and_clean_data(input_file, price_col_index=2)

    if df is not None:
        print(f"数据清洗完成，开始计算 {battery_capacity}kWh 电池的每日收益...")

        # 2. 计算策略
        # prominence=5 表示只有波动超过5块钱才会被视为波峰波谷，
        # 如果你想捕捉更细微的价差，可以把这个数改小（比如 2）
        strategy_df = calculate_daily_strategy(df, battery_kwh=battery_capacity, prominence=5)

        # 3. 打印预览
        print("\n--- 收益计算预览 (前5天) ---")
        print(strategy_df.head().to_string(index=False))

        # 4. 计算总收益
        total_profit = strategy_df['Total_Profit'].sum()
        print(f"\n>>> 累计总收益: {total_profit:.2f} EUR")

        # 5. 保存结果到 CSV
        strategy_df.to_csv(output_file, index=False, encoding='utf-8-sig') # utf-8-sig 防止Excel乱码
        print(f"\n结果已保存至: {output_file}")
    else:
        print("处理失败，请检查文件格式。")

In [ ]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
import os

# ==========================================
# 1. 数据清洗 (保持不变，已包含时间格式修复)
# ==========================================
def load_and_clean_data(file_path, price_col_index=2):
    try:
        df = pd.read_csv(file_path, sep=';', on_bad_lines='skip')
        clean_df = df.iloc[:, [0, price_col_index]].copy()
        clean_df.columns = ['RawTime', 'Price']

        # 使用指定格式解析，解决警告问题
        clean_df['Datetime'] = pd.to_datetime(clean_df['RawTime'], errors='coerce')
        clean_df = clean_df.dropna(subset=['Datetime'])

        if clean_df['Price'].dtype == object:
            clean_df['Price'] = clean_df['Price'].astype(str).str.replace(',', '.')
        clean_df['Price'] = pd.to_numeric(clean_df['Price'], errors='coerce')
        clean_df = clean_df.dropna(subset=['Price'])

        clean_df = clean_df.set_index('Datetime').sort_index()
        return clean_df
    except Exception as e:
        print(f"数据读取错误: {e}")
        return None

# ==========================================
# 2. 核心业务逻辑 (升级版：支持一天多次充放电)
# ==========================================
def calculate_daily_strategy(clean_df, battery_kwh=3, prominence=5):
    """
    输入: 清洗后的DataFrame (价格单位 €/MWh)
    输出: 包含所有交易策略的DataFrame
    """
    results = []
    grouped = clean_df.groupby(clean_df.index.date)

    for date, day_data in grouped:
        prices = day_data['Price'].values
        times = day_data.index

        if len(prices) < 5:
            continue

        # 1. 寻找当天的所有波峰波谷
        peak_indices, _ = find_peaks(prices, prominence=prominence, distance=1)
        valley_indices, _ = find_peaks(-prices, prominence=prominence, distance=1)

        # 2. 生成所有可能的“买-卖”候选对
        # 只要 卖出时间 > 买入时间 且 卖价 > 买价，就算一个潜在机会
        candidates = []
        for v_idx in valley_indices:
            for p_idx in peak_indices:
                if p_idx > v_idx: # 必须先买后卖
                    spread_mwh = prices[p_idx] - prices[v_idx]
                    if spread_mwh > 0: # 只看赚钱的
                        profit = (spread_mwh / 1000) * battery_kwh
                        candidates.append({
                            'buy_idx': v_idx,
                            'sell_idx': p_idx,
                            'spread_mwh': spread_mwh,
                            'profit': profit,
                            'buy_price': prices[v_idx],
                            'sell_price': prices[p_idx]
                        })

        # 3. 贪心算法选择最优且不冲突的组合
        # 按利润从高到低排序，优先锁定最赚的那一单
        candidates.sort(key=lambda x: x['profit'], reverse=True)

        selected_trades = []
        busy_intervals = [] # 记录电池被占用的时间段 (buy_idx, sell_idx)

        for trade in candidates:
            current_buy = trade['buy_idx']
            current_sell = trade['sell_idx']

            # 检查是否与已有交易的时间冲突
            # 冲突定义：当前交易的时间段与已选交易的时间段有重叠
            # 逻辑：Max(StartA, StartB) < Min(EndA, EndB) 即为重叠
            is_conflict = False
            for busy_start, busy_end in busy_intervals:
                # 严格冲突检查：在买入到卖出期间，电池被占用
                # 我们假设卖出后的瞬间电池变空，可以立刻进行下一次买入（也可以加缓冲时间）
                if max(current_buy, busy_start) < min(current_sell, busy_end):
                    is_conflict = True
                    break

            if not is_conflict:
                # 如果不冲突，接纳这单交易
                selected_trades.append(trade)
                # 标记这段时间被占用了
                busy_intervals.append((current_buy, current_sell))

        # 4. 如果当天没有交易，记录一行空数据
        if not selected_trades:
            results.append({
                'Date': date,
                'Charge_Time': 'No Action', 'Charge_Price_MWh': 0,
                'Discharge_Time': '-', 'Discharge_Price_MWh': 0,
                'Price_Spread_MWh': 0, 'Total_Profit_EUR': 0
            })
        else:
            # 5. 将选中的交易按时间先后排序并添加到结果中
            selected_trades.sort(key=lambda x: x['buy_idx'])

            for trade in selected_trades:
                results.append({
                    'Date': date,
                    'Charge_Time': times[trade['buy_idx']].strftime('%H:%M'),
                    'Charge_Price_MWh': trade['buy_price'],
                    'Discharge_Time': times[trade['sell_idx']].strftime('%H:%M'),
                    'Discharge_Price_MWh': trade['sell_price'],
                    'Price_Spread_MWh': trade['spread_mwh'],
                    'Total_Profit_EUR': trade['profit']
                })

    return pd.DataFrame(results)

# ==========================================
# 3. 主程序 (自动创建文件夹与保存)
# ==========================================
if __name__ == "__main__":
    # 配置区
    input_file = "data.csv"
    output_dir = r"/content" # 这里修改你想保存的文件夹路径
    output_filename = "battery_profit_report_multi_1.csv"

    # 自动处理路径
    if not os.path.exists(output_dir):
        try:
            os.makedirs(output_dir)
        except:
            output_dir = "." # 如果创建失败，保存到当前目录

    output_path = os.path.join(output_dir, output_filename)
    battery_capacity = 3  # 3 kWh

    print("--- 开始处理 (支持一日多次充放电) ---")

    df = load_and_clean_data(input_file, price_col_index=2)

    if df is not None:
        # 计算策略
        strategy_df = calculate_daily_strategy(df, battery_kwh=battery_capacity, prominence=1)

        # 打印部分结果预览
        print("\n--- 策略预览 (前10条交易记录) ---")
        if not strategy_df.empty:
            print(strategy_df.head(10).to_string(index=False))

        # 计算总收益
        total_profit = strategy_df['Total_Profit_EUR'].sum()

        # 计算交易总次数
        # 过滤掉 "No Action" 的行
        valid_trades = strategy_df[strategy_df['Total_Profit_EUR'] > 0]
        invalid_trades = strategy_df[strategy_df['Total_Profit_EUR'] <= 0]
        trade_count = len(valid_trades)
        miss_count = len(invalid_trades)

        print(f"\n{'='*40}")
        print(f" 数据分析报告")
        print(f"{'='*40}")
        print(f" 总交易次数 : {trade_count} 次")
        print(f"  错过 : {miss_count} 次")
        print(f" 累计总收益 : {total_profit:.4f} EUR")
        print(f" 平均每单收益: {(total_profit/trade_count if trade_count>0 else 0):.4f} EUR")
        print(f"{'='*40}")

        # 保存CSV
        strategy_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"\n✅ 结果已保存至: {output_path}")

--- 开始处理 (支持一日多次充放电) ---


/tmp/ipython-input-1456344685.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  clean_df['Datetime'] = pd.to_datetime(clean_df['RawTime'], errors='coerce')



--- 策略预览 (前10条交易记录) ---
      Date Charge_Time  Charge_Price_MWh Discharge_Time  Discharge_Price_MWh  Price_Spread_MWh  Total_Profit_EUR
2024-10-31       03:00             94.81          07:00               135.53             40.72           0.12216
2024-10-31       12:00             88.91          17:00               129.00             40.09           0.12027
2024-11-01       03:00             54.54          07:00                70.35             15.81           0.04743
2024-11-01       12:00             29.49          17:00               119.87             90.38           0.27114
2024-11-02       03:00             86.00          17:00               158.56             72.56           0.21768
2024-11-03       03:00             85.09          07:00                98.11             13.02           0.03906
2024-11-03       12:00             52.63          17:00               127.91             75.28           0.22584
2024-11-04       03:00             89.86          17:00               2